In [4]:
import numpy as np
import pandas as pd
import midi
import os

In [6]:
pattern = midi.read_midifile("./train/mz_311_1.mid")
print (pattern)

midi.Pattern(format=1, resolution=480, tracks=\
[midi.Track(\
  [midi.TrackNameEvent(tick=0, text='Klaviersonate Nr. 8 KV 311 1. Satz', data=[75, 108, 97, 118, 105, 101, 114, 115, 111, 110, 97, 116, 101, 32, 78, 114, 46, 32, 56, 32, 75, 86, 32, 51, 49, 49, 32, 49, 46, 32, 83, 97, 116, 122]),
   midi.CopyrightMetaEvent(tick=0, text='Copyright © 2006 von Bernd Krüger. ', data=[67, 111, 112, 121, 114, 105, 103, 104, 116, 32, 169, 32, 50, 48, 48, 54, 32, 118, 111, 110, 32, 66, 101, 114, 110, 100, 32, 75, 114, 252, 103, 101, 114, 46, 32]),
   midi.TextMetaEvent(tick=0, text='Wolfgang Amadeus Mozart', data=[87, 111, 108, 102, 103, 97, 110, 103, 32, 65, 109, 97, 100, 101, 117, 115, 32, 77, 111, 122, 97, 114, 116]),
   midi.TextMetaEvent(tick=0, text='Herausgegeben am  1.7.2006\n', data=[72, 101, 114, 97, 117, 115, 103, 101, 103, 101, 98, 101, 110, 32, 97, 109, 32, 32, 49, 46, 55, 46, 50, 48, 48, 54, 10]),
   midi.TextMetaEvent(tick=0, text='Update am 5.6.2013\n', data=[85, 112, 100, 97, 116, 

In [5]:
# read midi file
def read_midi(filedir):
    result=[]
    for filename in os.listdir(filedir):
        filepath=filedir+"/"+filename
        pattern = midi.read_midifile(filepath)
        result.append(pattern)
    return result

In [6]:
midi_list=read_midi("./train")

# Let's make some music

In [8]:
midi_list[0][1]

midi.Track(\
  [midi.TrackNameEvent(tick=0, text='Piano right', data=[80, 105, 97, 110, 111, 32, 114, 105, 103, 104, 116]),
   midi.ProgramChangeEvent(tick=0, channel=0, data=[0]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[7, 100]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[10, 64]),
   midi.NoteOnEvent(tick=0, channel=0, data=[69, 60]),
   midi.NoteOnEvent(tick=0, channel=0, data=[74, 73]),
   midi.TextMetaEvent(tick=0, text='bdca426d104a26ac9dcb070447587523', data=[98, 100, 99, 97, 52, 50, 54, 100, 49, 48, 52, 97, 50, 54, 97, 99, 57, 100, 99, 98, 48, 55, 48, 52, 52, 55, 53, 56, 55, 53, 50, 51]),
   midi.ControlChangeEvent(tick=0, channel=0, data=[91, 127]),
   midi.NoteOnEvent(tick=0, channel=0, data=[66, 60]),
   midi.NoteOnEvent(tick=480, channel=0, data=[66, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[74, 0]),
   midi.NoteOnEvent(tick=0, channel=0, data=[69, 0]),
   midi.NoteOnEvent(tick=480, channel=0, data=[79, 66]),
   midi.NoteOnEvent(tick=120, channe

In [115]:
def get_note_event(midi_file):
    note_event=[]
    
    track_total=len(midi_file)
    for track_num in range(track_total):
        cur_track=midi_file[track_num]
        event_total=len(cur_track)
        for event_num in range(event_total):
            cur_event=cur_track[event_num]
            if type(cur_event)==midi.events.NoteOnEvent:
                note_event.append(cur_event)
    return note_event

In [152]:
def get_note_info(note_event):
    total_info=[]
    for event_idx in range(len(note_event)):
        cur_info=[]
        cur_info.append(note_event[event_idx].get_pitch())
        #cur_info.append(note_event[event_idx].get_velocity())
        total_info.append(np.array(cur_info))
    return np.array(total_info)

In [130]:
def make_note_data(midi_list):
    X=[]
    for midi_idx in range(len(midi_list)):
        midi_file=midi_list[midi_idx]
        note_event=get_note_event(midi_file)
        note_info=get_note_info(note_event)
        X.append(note_info)
    return np.array(X)

In [153]:
train_data=make_note_data(midi_list)

# LSTM 

In [172]:
def normalize(data):
    minx=np.min(data)
    maxx=np.max(data)
    if maxx==minx:
        result=[0 for element in data]
    result=[(element-minx)/(maxx-minx) for element in data]
    result=np.array(result)
    return result,[minx,maxx]

In [170]:
def make_LSTM_XY(train_data):
    timestep=20
    X=[]
    Y=[]
    tr_info=[]
    for cur_data in train_data:
        total_len=len(cur_data)
        for note_start in range(total_len-timestep):
            pre_note=cur_data[note_start:note_start+timestep+1]
            post_note,tr=normalize(pre_note)
            X.append(np.array(post_note[0:timestep]))
            Y.append(np.array(post_note[timestep]))
            tr_info.append(tr)
    return np.array(X), np.array(Y), np.array(tr_info)

In [171]:
trainX,trainY,train_tr=make_LSTM_XY(train_data[0:3])
testX,testY,test_tr=make_LSTM_XY(train_data[3:5])

C:\Users\crit9\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


In [177]:
from keras.models import Sequential
from keras.layers import GRU, Bidirectional, LSTM, Dropout, Dense

In [178]:
# Make LSTM network
cell_num=48
dense_num=128
drop_per=0.3

model=Sequential()
model.add(Bidirectional(GRU(cell_num,return_sequences=True),input_shape=(20, 1)))
model.add(Bidirectional(GRU(cell_num)))
model.add(Dropout(drop_per))
model.add(Dense(dense_num,activation='sigmoid'))
model.add(Dense(1,activation='linear'))

In [179]:
# Training
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=10, batch_size=500, verbose=2, shuffle=False)

Epoch 1/10
31s - loss: 0.4410
Epoch 2/10
26s - loss: 0.1225
Epoch 3/10
27s - loss: 0.1132
Epoch 4/10
27s - loss: 0.1095
Epoch 5/10
26s - loss: 0.1079
Epoch 6/10
26s - loss: 0.1071
Epoch 7/10
25s - loss: 0.1049
Epoch 8/10
25s - loss: 0.1018
Epoch 9/10
25s - loss: 0.0962
Epoch 10/10
27s - loss: 0.0926


# Note prediction

In [181]:
def translate(pred,info):
    temp_result=[]
    # transform each Y
    for i in range(len(pred)):
        cur_info=info[i]
        cur_pred=pred[i]
        minx=cur_info[0]
        maxx=cur_info[1]
        xrange=maxx-minx
        cur_result=[(p*xrange)+minx for p in cur_pred]
        temp_result.append(np.array(cur_result))
    result=[]
    for i in range(len(temp_result)-1,-1,-1):
        if i==0:
            cur_result=np.flip(temp_result[i],0)
            result.extend(cur_result)
        else:
            result.append(temp_result[i][-1])
    result=np.array(result)
    return np.flip(result,0)

In [183]:
pred_t=model.predict(trainX)
pred_v=model.predict(testX)
pred_train=translate(pred_t,train_tr)
pred_test=translate(pred_v,test_tr)

In [184]:
train_result=np.round(pred_train)
test_result=np.round(pred_test)

In [186]:
print (train_result[0])

78.0


In [190]:
# make midi file
def make_midi(result):
    tempo=120
    dur=60
    pattern = midi.Pattern()
    track = midi.Track()
    pattern.append(track)
    for pitch in result:
        on = midi.NoteOnEvent(tick=tempo, velocity=dur, data=[int(pitch),dur])
        track.append(on)
    eot = midi.EndOfTrackEvent(tick=1)
    track.append(eot)
    return pattern

In [188]:
train_midi=make_midi(train_result)
test_midi=make_midi(test_result)

midi.Pattern(format=1, resolution=220, tracks=\
[midi.Track(\
  [midi.NoteOnEvent(tick=120, channel=0, data=[78, 60]),
   midi.NoteOnEvent(tick=120, channel=0, data=[78, 60]),
   midi.NoteOnEvent(tick=120, channel=0, data=[79, 60]),
   midi.NoteOnEvent(tick=120, channel=0, data=[76, 60]),
   midi.NoteOnEvent(tick=120, channel=0, data=[80, 60]),
   midi.NoteOnEvent(tick=120, channel=0, data=[77, 60]),
   midi.NoteOnEvent(tick=120, channel=0, data=[82, 60]),
   midi.NoteOnEvent(tick=120, channel=0, data=[82, 60]),
   midi.NoteOnEvent(tick=120, channel=0, data=[83, 60]),
   midi.NoteOnEvent(tick=120, channel=0, data=[84, 60]),
   midi.NoteOnEvent(tick=120, channel=0, data=[84, 60]),
   midi.NoteOnEvent(tick=120, channel=0, data=[85, 60]),
   midi.NoteOnEvent(tick=120, channel=0, data=[84, 60]),
   midi.NoteOnEvent(tick=120, channel=0, data=[83, 60]),
   midi.NoteOnEvent(tick=120, channel=0, data=[84, 60]),
   midi.NoteOnEvent(tick=120, channel=0, data=[81, 60]),
   midi.NoteOnEvent(tick=1

ValueError: cannot convert float NaN to integer

In [189]:
midi.write_midifile("train.mid", train_midi)
midi.write_midifile("test.mid", test_midi)

AttributeError: 'NoneType' object has no attribute 'format'